In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

---

## Capstone Project: GPU-Accelerated Image Convolution

### Project Requirements

Build a complete image convolution system that:
1. Loads images from disk
2. Applies various filters (blur, sharpen, edge detect)
3. Uses optimized CUDA kernels
4. Demonstrates performance gains over CPU
5. Uses streams for overlapped processing of multiple images

### Skills Demonstrated

```
✓ Week 1-2: Thread indexing, 2D grids
✓ Week 3-4: Shared memory, memory coalescing
✓ Week 5:   Scan for histogram normalization
✓ Week 6:   2D tiled algorithms
✓ Week 7:   Occupancy and memory optimization
✓ Week 8:   Profiling and roofline analysis
✓ Week 9:   Streams for multi-image processing
✓ Week 10:  CUDA Graphs for repeated filters
✓ Week 11:  Cooperative groups for reductions
✓ Week 12:  Multi-GPU for large images
```

---

## Part 1: Core Convolution Kernel

### CUDA C++ Implementation (Primary)

```cpp
// convolution.cu - Complete image convolution system
#include <stdio.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 16
#define FILTER_RADIUS 2
#define FILTER_SIZE (2 * FILTER_RADIUS + 1)

// Constant memory for filter coefficients
__constant__ float c_filter[FILTER_SIZE * FILTER_SIZE];

// ============================================================
// Naive Convolution (Baseline)
// ============================================================
__global__ void convNaive(
    float* output, const float* input,
    int width, int height
) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x >= width || y >= height) return;
    
    float sum = 0.0f;
    
    for (int fy = -FILTER_RADIUS; fy <= FILTER_RADIUS; fy++) {
        for (int fx = -FILTER_RADIUS; fx <= FILTER_RADIUS; fx++) {
            int ix = min(max(x + fx, 0), width - 1);
            int iy = min(max(y + fy, 0), height - 1);
            
            int fidx = (fy + FILTER_RADIUS) * FILTER_SIZE + 
                       (fx + FILTER_RADIUS);
            
            sum += input[iy * width + ix] * c_filter[fidx];
        }
    }
    
    output[y * width + x] = sum;
}

// ============================================================
// Optimized Convolution with Shared Memory
// ============================================================
__global__ void convShared(
    float* output, const float* input,
    int width, int height
) {
    // Shared memory tile with apron
    const int TILE_W = BLOCK_SIZE + 2 * FILTER_RADIUS;
    __shared__ float smem[TILE_W][TILE_W];
    
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int x = blockIdx.x * BLOCK_SIZE + tx;
    int y = blockIdx.y * BLOCK_SIZE + ty;
    
    // Calculate source positions including apron
    int srcX = x - FILTER_RADIUS;
    int srcY = y - FILTER_RADIUS;
    
    // Load main tile element
    if (srcX >= 0 && srcX < width && srcY >= 0 && srcY < height) {
        smem[ty][tx] = input[srcY * width + srcX];
    } else {
        smem[ty][tx] = 0.0f;
    }
    
    // Load additional elements for apron (right and bottom edges)
    if (tx < 2 * FILTER_RADIUS) {
        int ax = srcX + BLOCK_SIZE;
        if (ax >= 0 && ax < width && srcY >= 0 && srcY < height) {
            smem[ty][tx + BLOCK_SIZE] = input[srcY * width + ax];
        } else {
            smem[ty][tx + BLOCK_SIZE] = 0.0f;
        }
    }
    
    if (ty < 2 * FILTER_RADIUS) {
        int ay = srcY + BLOCK_SIZE;
        if (srcX >= 0 && srcX < width && ay >= 0 && ay < height) {
            smem[ty + BLOCK_SIZE][tx] = input[ay * width + srcX];
        } else {
            smem[ty + BLOCK_SIZE][tx] = 0.0f;
        }
    }
    
    if (tx < 2 * FILTER_RADIUS && ty < 2 * FILTER_RADIUS) {
        int ax = srcX + BLOCK_SIZE;
        int ay = srcY + BLOCK_SIZE;
        if (ax >= 0 && ax < width && ay >= 0 && ay < height) {
            smem[ty + BLOCK_SIZE][tx + BLOCK_SIZE] = input[ay * width + ax];
        } else {
            smem[ty + BLOCK_SIZE][tx + BLOCK_SIZE] = 0.0f;
        }
    }
    
    __syncthreads();
    
    // Compute convolution from shared memory
    if (x < width && y < height) {
        float sum = 0.0f;
        
        #pragma unroll
        for (int fy = 0; fy < FILTER_SIZE; fy++) {
            #pragma unroll
            for (int fx = 0; fx < FILTER_SIZE; fx++) {
                sum += smem[ty + fy][tx + fx] * 
                       c_filter[fy * FILTER_SIZE + fx];
            }
        }
        
        output[y * width + x] = sum;
    }
}
```

---

## Part 2: Filter Definitions and Host Code

```cpp
// ============================================================
// Filter Definitions
// ============================================================

void createGaussianBlur(float* filter) {
    float kernel[] = {
        1.0f/256, 4.0f/256,  7.0f/256,  4.0f/256, 1.0f/256,
        4.0f/256, 16.0f/256, 26.0f/256, 16.0f/256, 4.0f/256,
        7.0f/256, 26.0f/256, 41.0f/256, 26.0f/256, 7.0f/256,
        4.0f/256, 16.0f/256, 26.0f/256, 16.0f/256, 4.0f/256,
        1.0f/256, 4.0f/256,  7.0f/256,  4.0f/256, 1.0f/256
    };
    memcpy(filter, kernel, FILTER_SIZE * FILTER_SIZE * sizeof(float));
}

void createSharpen(float* filter) {
    float kernel[] = {
         0,  0, -1,  0,  0,
         0, -1, -1, -1,  0,
        -1, -1, 13, -1, -1,
         0, -1, -1, -1,  0,
         0,  0, -1,  0,  0
    };
    memcpy(filter, kernel, FILTER_SIZE * FILTER_SIZE * sizeof(float));
}

void createEdgeDetect(float* filter) {
    float kernel[] = {
        -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,
        -1, -1, 24, -1, -1,
        -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1
    };
    memcpy(filter, kernel, FILTER_SIZE * FILTER_SIZE * sizeof(float));
}

// ============================================================
// Stream-based Multi-Image Processing
// ============================================================

void processMultipleImages(
    float** h_inputs, float** h_outputs,
    int numImages, int width, int height
) {
    const int NUM_STREAMS = 4;
    size_t imageBytes = width * height * sizeof(float);
    
    cudaStream_t streams[NUM_STREAMS];
    float* d_input[NUM_STREAMS];
    float* d_output[NUM_STREAMS];
    
    // Create streams and allocate device memory
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
        cudaMalloc(&d_input[i], imageBytes);
        cudaMalloc(&d_output[i], imageBytes);
    }
    
    dim3 block(BLOCK_SIZE, BLOCK_SIZE);
    dim3 grid((width + BLOCK_SIZE - 1) / BLOCK_SIZE,
              (height + BLOCK_SIZE - 1) / BLOCK_SIZE);
    
    // Process images with overlapping
    for (int img = 0; img < numImages; img++) {
        int streamIdx = img % NUM_STREAMS;
        
        // H2D
        cudaMemcpyAsync(d_input[streamIdx], h_inputs[img],
                        imageBytes, cudaMemcpyHostToDevice,
                        streams[streamIdx]);
        
        // Compute
        convShared<<<grid, block, 0, streams[streamIdx]>>>(
            d_output[streamIdx], d_input[streamIdx], width, height);
        
        // D2H
        cudaMemcpyAsync(h_outputs[img], d_output[streamIdx],
                        imageBytes, cudaMemcpyDeviceToHost,
                        streams[streamIdx]);
    }
    
    // Sync all streams
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamSynchronize(streams[i]);
    }
    
    // Cleanup
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
        cudaFree(d_input[i]);
        cudaFree(d_output[i]);
    }
}
```

---

## Part 3: CUDA Graph for Repeated Filters

```cpp
// ============================================================
// CUDA Graph for Multi-Pass Filtering
// ============================================================

void multiPassFilter(
    float* d_output, float* d_input,
    float* d_temp,
    int width, int height,
    int numPasses
) {
    dim3 block(BLOCK_SIZE, BLOCK_SIZE);
    dim3 grid((width + BLOCK_SIZE - 1) / BLOCK_SIZE,
              (height + BLOCK_SIZE - 1) / BLOCK_SIZE);
    
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    
    cudaGraph_t graph;
    cudaGraphExec_t instance;
    
    // Capture graph for one pass
    cudaStreamBeginCapture(stream, cudaStreamCaptureModeGlobal);
    
    // Forward pass: input -> temp
    convShared<<<grid, block, 0, stream>>>(
        d_temp, d_input, width, height);
    
    // Backward pass: temp -> output
    convShared<<<grid, block, 0, stream>>>(
        d_output, d_temp, width, height);
    
    cudaStreamEndCapture(stream, &graph);
    cudaGraphInstantiate(&instance, graph, NULL, NULL, 0);
    
    // Execute graph multiple times
    for (int pass = 0; pass < numPasses; pass++) {
        cudaGraphLaunch(instance, stream);
    }
    
    cudaStreamSynchronize(stream);
    
    cudaGraphExecDestroy(instance);
    cudaGraphDestroy(graph);
    cudaStreamDestroy(stream);
}
```

---

## Part 4: Main Program and Benchmarking

```cpp
// ============================================================
// Main Program
// ============================================================

int main(int argc, char** argv) {
    const int WIDTH = 4096;
    const int HEIGHT = 4096;
    size_t imageBytes = WIDTH * HEIGHT * sizeof(float);
    
    // Allocate host memory (pinned for async transfers)
    float *h_input, *h_output;
    cudaMallocHost(&h_input, imageBytes);
    cudaMallocHost(&h_output, imageBytes);
    
    // Initialize with test pattern
    for (int i = 0; i < WIDTH * HEIGHT; i++) {
        h_input[i] = (float)(i % 256) / 255.0f;
    }
    
    // Allocate device memory
    float *d_input, *d_output;
    cudaMalloc(&d_input, imageBytes);
    cudaMalloc(&d_output, imageBytes);
    
    // Create and upload Gaussian blur filter
    float h_filter[FILTER_SIZE * FILTER_SIZE];
    createGaussianBlur(h_filter);
    cudaMemcpyToSymbol(c_filter, h_filter, 
                       FILTER_SIZE * FILTER_SIZE * sizeof(float));
    
    // Copy input to device
    cudaMemcpy(d_input, h_input, imageBytes, cudaMemcpyHostToDevice);
    
    // Setup kernel launch parameters
    dim3 block(BLOCK_SIZE, BLOCK_SIZE);
    dim3 grid((WIDTH + BLOCK_SIZE - 1) / BLOCK_SIZE,
              (HEIGHT + BLOCK_SIZE - 1) / BLOCK_SIZE);
    
    // Warmup
    convShared<<<grid, block>>>(d_output, d_input, WIDTH, HEIGHT);
    cudaDeviceSynchronize();
    
    // =========================================
    // Benchmark: Naive vs Optimized
    // =========================================
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    const int RUNS = 100;
    float ms;
    
    // Naive kernel
    cudaEventRecord(start);
    for (int i = 0; i < RUNS; i++) {
        convNaive<<<grid, block>>>(d_output, d_input, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);
    
    float naiveTime = ms / RUNS;
    printf("Naive convolution:     %.2f ms\n", naiveTime);
    
    // Optimized kernel
    cudaEventRecord(start);
    for (int i = 0; i < RUNS; i++) {
        convShared<<<grid, block>>>(d_output, d_input, WIDTH, HEIGHT);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&ms, start, stop);
    
    float optTime = ms / RUNS;
    printf("Optimized convolution: %.2f ms\n", optTime);
    printf("Speedup: %.2fx\n", naiveTime / optTime);
    
    // Calculate effective bandwidth
    float bandwidth = 2.0f * imageBytes / (optTime / 1000.0f) / 1e9;
    printf("Effective bandwidth:   %.2f GB/s\n", bandwidth);
    
    // Cleanup
    cudaFree(d_input);
    cudaFree(d_output);
    cudaFreeHost(h_input);
    cudaFreeHost(h_output);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    printf("\nCapstone project complete!\n");
    return 0;
}

// Compile: nvcc -O3 -arch=sm_70 convolution.cu -o convolution
// Profile: ncu --set full ./convolution
```

---

## Capstone Extensions (Optional Challenges)

### Extension 1: Multi-GPU Large Image Processing
- Split large images across GPUs
- Handle halo exchange for overlapping regions

### Extension 2: Real-time Video Filtering
- Process video frames in a pipeline
- Use streams for frame-level parallelism

### Extension 3: Separable Filters
- Implement separable convolution (2 1D passes)
- Compare performance with 2D convolution

### Extension 4: cuDNN Comparison
- Compare your implementation with cuDNN
- Analyze where cuDNN does better

---

## 🎉 Curriculum Complete!

```
╔══════════════════════════════════════════════════════════════╗
║                                                              ║
║          🎓 CONGRATULATIONS! 🎓                              ║
║                                                              ║
║   You have completed the 12-week CUDA curriculum!           ║
║                                                              ║
║   Skills Mastered:                                          ║
║   ✓ GPU Architecture & Programming Model                    ║
║   ✓ Thread Indexing & Memory Hierarchies                    ║
║   ✓ Shared Memory & Synchronization                         ║
║   ✓ Parallel Patterns (Reduce, Scan, Stencil)               ║
║   ✓ Memory Optimization & Coalescing                        ║
║   ✓ Occupancy & Performance Analysis                        ║
║   ✓ Streams & Asynchronous Execution                        ║
║   ✓ CUDA Graphs                                             ║
║   ✓ Cooperative Groups & Dynamic Parallelism                ║
║   ✓ Multi-GPU Programming                                   ║
║   ✓ Profiling with Nsight                                   ║
║                                                              ║
║   Next Steps:                                                ║
║   • Build real projects with GPU acceleration               ║
║   • Explore CUDA libraries (cuBLAS, cuDNN, Thrust)          ║
║   • Learn about tensor cores and mixed precision            ║
║   • Contribute to open-source GPU projects                  ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
```